In [28]:
import os
from dotenv import load_dotenv
from tools.VCL_query_timeseries import timeseries_query
from tools.VCL_query_timeseries_area import timeseries_area_query
from tools.VCL_get_query_status import get_query_status
from tools.VCL_get_query_result import get_query_result
from tools.VCL_query_aggregation import construct_aggregation_dict, aggregation_query
from json import JSONDecodeError
from time import sleep
import pandas as pd

load_dotenv()
api_key = os.environ["API_SECRET_KEY"]


In [90]:
stuff = get_query_result("53fcc1fa-f966-4bba-8770-a282f43ebff6", api_key, 10000000, 1)
stuff = stuff.json()

In [91]:
len(stuff)

5720280

In [92]:
import pandas as pd
df = pd.DataFrame.from_records(stuff, index=range(len(stuff)))

df.to_csv(f"area_wind_speed_data_bigger5.csv", mode="a", index=False)

In [5]:
### AGGREGATION QUERY
min_longitude = -4.4454
max_longitude = 12.0059
min_latitude = 53.9954
max_latitude = 56.9954
start_date = "20120101"
end_date = "20121231"
agg_functions = ["AVG", "MAX", "MIN", "SD"] # At least one function is required

vars_3d = ["xland"]
vars_4d = ["wsp"]
heights = [150] 
aggregation = construct_aggregation_dict(agg_functions, vars_3d, vars_4d, heights)

response = aggregation_query(min_longitude, max_longitude, min_latitude, max_latitude, start_date, end_date, aggregation, api_key)
print("Aggregation Query: ")
try:
    response_json = response.json()
    print(response_json)
except JSONDecodeError:
    print("error:", end="\t")
    print(response.text)

Aggregation Query: 
Timeseries Area Query: 
{'queryId': '1ad06c41-c651-4919-8750-7ab159018207'}


In [ ]:
# 2022
# min_latitude = 50.9954, max_latitude = 53.9954
# {'queryId': '6a117fc4-030a-4584-8a3b-485d28f520ee'}
# min_latitude = 53.9954, max_latitude = 56.9954
# {'queryId': 'fee7d638-2254-412f-9cd0-60a9576c184a'}

# 2012
# min_latitude = 50.9954, max_latitude = 53.9954
# {'queryId': '0fcc4e59-bf75-4cd0-a52c-2ed46ddad3a0'}
# min_latitude = 53.9954, max_latitude = 56.9954
# {'queryId': '1ad06c41-c651-4919-8750-7ab159018207'}

In [23]:
###########
#### POINT REQUEST
##########

latitude = 53.62
longitude = 4.42
start_date = "2012-01-01"
end_date = "2022-12-31"
vars_4d = ["wsp","wdir", "rho"] # Can be empty []
vars_3d = ["xland"]
heights = [150] # Must not be empty if vars_4d is not empty


response = timeseries_query(latitude, longitude, start_date, end_date, vars_4d, vars_3d, heights, api_key)
try:
    response_json = response.json()
    query_id = response_json["queryId"]
    print("Point Query: ")
    print(response_json)
except JSONDecodeError:
    print("error:")
    print(response.text)

Point Query: 
{'queryId': 'b90b21cd-2035-4148-a359-99a867988a3f'}


# Area Queries

min_longitude = 0.98<br>
max_longitude = 4.2<br>
latitudes from 51.05 until 54.8

min_longitude = -1.5<br>
max_longitude = 0.98<br>
latitudes from 51.05 until 54.8

min_longitude = 4.2<br>
max_longitude = 8.2<br>
latitudes from 51.05 until 54.8

In [108]:
###########
#### AREA REQUEST
##########

min_longitude = 4.2
max_longitude = 8.2

start_date = "20220101"
end_date = "20221231"
vars_4d = ["wsp","rho"]
vars_3d = ["xland"]
heights = [150]


max_step_size = 0.3
from_lat = 51.349999999999994
to_lat = 54.8
min_latitude, max_latitude = from_lat, min(to_lat, from_lat + max_step_size)

# query_list = []

while True:
    response = timeseries_area_query(min_longitude, max_longitude, min_latitude, max_latitude, start_date, end_date, vars_4d, vars_3d, heights, api_key)
    try:
        response_json = response.json()
    except JSONDecodeError:
        print("error:")
        print(response.text)
    else:
        print(f"{min_latitude=}, {max_latitude=} - {response_json}")
        min_latitude = max_latitude
        max_latitude = min(to_lat, max_latitude + max_step_size)
        query_list.append({
            "query_id": response_json["queryId"],
            "min_latitude": min_latitude,
            "max_latitude": max_latitude,
        })
        
        
    
    if max_latitude >= to_lat:
        break
    sleep(60)



min_latitude=51.349999999999994, max_latitude=51.64999999999999 - {'queryId': 'f3d13c39-bd0e-4c88-bc63-c3ea350c4e8a'}
min_latitude=51.64999999999999, max_latitude=51.94999999999999 - {'queryId': 'd5cc0dd5-ebfb-45b5-89c4-05e974eb1b43'}
min_latitude=51.94999999999999, max_latitude=52.249999999999986 - {'queryId': '5505432a-9928-42df-8cbf-0b2e0af50c57'}
min_latitude=52.249999999999986, max_latitude=52.54999999999998 - {'queryId': '003654b0-99e2-4610-9ee2-582a292fafe5'}
min_latitude=52.54999999999998, max_latitude=52.84999999999998 - {'queryId': '2a64cb08-05dc-4628-b227-20041b10ddce'}
error:
Request limit is reached.


KeyboardInterrupt: 

In [109]:
query_list

[{'query_id': '6c6326c3-387d-463d-8252-dff6c301beee',
  'min_latitude': 51.3,
  'max_latitude': 51.3},
 {'query_id': 'e04efd37-0e53-4de4-a20d-22bbea2028b4',
  'min_latitude': 51.3,
  'max_latitude': 51.55},
 {'query_id': '554cdede-4274-406b-ae79-1316b2894adc',
  'min_latitude': 51.55,
  'max_latitude': 51.8},
 {'query_id': '9f2401c9-614d-4922-b73e-024fbefea181',
  'min_latitude': 51.8,
  'max_latitude': 52.05},
 {'query_id': 'cfa84291-0ca1-4ca7-88a4-b50f7b4f7072',
  'min_latitude': 52.05,
  'max_latitude': 52.3},
 {'query_id': '8f67e7eb-82a0-4220-9bf6-971aa22e171b',
  'min_latitude': 52.3,
  'max_latitude': 52.55},
 {'query_id': '13238ed3-bda1-45fd-a05c-2824320c1751',
  'min_latitude': 52.599999999999994,
  'max_latitude': 52.89999999999999},
 {'query_id': '1a3ba5a6-e121-4043-8bae-8e3e5bbe8bfe',
  'min_latitude': 52.89999999999999,
  'max_latitude': 53.19999999999999},
 {'query_id': '8ebc6013-4fb5-4ba1-9c66-5a589d4f5425',
  'min_latitude': 53.19999999999999,
  'max_latitude': 53.499999

In [114]:
def check_for_completed_query(query_info: dict, query_list: list, completed_queries: list) -> bool:
    response = get_query_status(query_id, api_key)
    try:
        response_json = response.json()
    except Exception:
        print(response.text)
        query_list.append(query_info)
    else:
        if response_json.get("currentState", None) == "Finished":
            completed_queries.append(query_info)
            print(f"Finished: {query_info}")
            return True
        else:
            query_list.append(query_info)
            if response_json.get("currentState") is not None:
                print(f"{response_json.get('currentState')}: {query_info}")
                return False
            else:
                print(f"{response_json}: {query_info}")
                return False

completed_queries = []
downloaded_queries = []
while len(query_list) > 0 or len(completed_queries) > 0:
    
    if len(completed_queries) == 0:
        query_info = query_list.pop(0)
        print("Checking for completion:")
        if not check_for_completed_query(query_info, query_list, completed_queries):
            sleep(60)
    else:
        query_info = completed_queries.pop(0)
        stuff = get_query_result(query_info["query_id"], api_key, 10000000, 1)
        try:
            stuff = stuff.json()
        except Exception:
            print(f"Failed download {query_info}")
            completed_queries.append(query_info)
        if len(stuff) == 1:
            print(f"failed download: {query_info}")
            completed_queries.append(query_info)
            sleep(60)
        else:
            df = pd.DataFrame.from_records(stuff, index=range(len(stuff)))
            df.to_csv(f"area_wind_speed_data_bigger{query_info['query_id'][:4]}.csv", mode="a", index=False)
            downloaded_queries.append(query_info)
            print(f"downloaded ({len(stuff)}) area_wind_speed_data_bigger{[query_info['query_id'][:4]]}.csv")
            

Checking for completion:
Finished: {'query_id': 'e04efd37-0e53-4de4-a20d-22bbea2028b4', 'min_latitude': 51.3, 'max_latitude': 51.55}
downloaded (4388760) area_wind_speed_data_bigger['e04e'].csv
Checking for completion:
Finished: {'query_id': '554cdede-4274-406b-ae79-1316b2894adc', 'min_latitude': 51.55, 'max_latitude': 51.8}
downloaded (4309920) area_wind_speed_data_bigger['554c'].csv
Checking for completion:
Finished: {'query_id': '9f2401c9-614d-4922-b73e-024fbefea181', 'min_latitude': 51.8, 'max_latitude': 52.05}
